In [1]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import log10, floor
import os

from relational_rnn_models import RelationalMemoryGenerator

In [2]:
# Define variables
n_vars = 3
var_ids = list(range(n_vars))
var_names = ['var' + str(i) for i in var_ids]
var_weights = [0.1, 0.6, 0.3] # variable distribution of mock data
n_time_steps = 6
n_individuals = 1000

noise_length = 2

In [3]:
# Helper function(s)

# round a number to n significant digits
def round_to_n(x, n = 2):
    return round(x, -int(floor(log10(abs(x)))) + (n - 1)) if x != 0 else 0

# visualize the output of the generator
def visualize_output(generator, z, n = 2):
    p = generator(z).view(n_time_steps, n_vars)
    p.shape
    for t in range(p.shape[0]):
        tmp = []
        for f in range(p.shape[1]):
            tmp.append(round_to_n(p[t,f], n))
        print(tmp)

In [4]:
# Generate mock data

events = []

start_time = time.time()

for indv in range(n_individuals):
    tmp = []
    for t in range(n_time_steps):
        if t > 0 and tmp[t - 1] == 'var2':
            weights = [0.7, 0.2, 0.1]
            var = np.random.choice(var_names, p=weights)
        else:
            var = np.random.choice(var_names, p=var_weights)
        tmp.append(var)
    events.append(tmp)
        
print('time taken:', round_to_n(time.time() - start_time), 'seconds')

for i in range(10):
    print(events[i])

time taken: 0.12 seconds
['var0', 'var2', 'var2', 'var0', 'var1', 'var1']
['var1', 'var1', 'var1', 'var1', 'var1', 'var2']
['var1', 'var1', 'var0', 'var1', 'var1', 'var1']
['var2', 'var0', 'var1', 'var2', 'var1', 'var2']
['var2', 'var0', 'var1', 'var1', 'var0', 'var2']
['var2', 'var0', 'var2', 'var0', 'var2', 'var0']
['var2', 'var0', 'var2', 'var0', 'var2', 'var0']
['var1', 'var1', 'var2', 'var0', 'var1', 'var1']
['var2', 'var0', 'var1', 'var1', 'var0', 'var2']
['var0', 'var1', 'var0', 'var1', 'var1', 'var1']


In [5]:
vars_to_indices = dict([(v, i) for i, v in enumerate(var_names)])
print(vars_to_indices)
data = torch.tensor([[vars_to_indices[e] for e in event] for event in events])
data[:10]

{'var0': 0, 'var1': 1, 'var2': 2}


tensor([[0, 2, 2, 0, 1, 1],
        [1, 1, 1, 1, 1, 2],
        [1, 1, 0, 1, 1, 1],
        [2, 0, 1, 2, 1, 2],
        [2, 0, 1, 1, 0, 2],
        [2, 0, 2, 0, 2, 0],
        [2, 0, 2, 0, 2, 0],
        [1, 1, 2, 0, 1, 1],
        [2, 0, 1, 1, 0, 2],
        [0, 1, 0, 1, 1, 1]])

In [6]:
# Test generator output

mem_slots = 4
head_size = 2
embed_size = 2
num_tokens = n_vars
temperature = 1
num_heads = 1

G = RelationalMemoryGenerator(mem_slots, head_size, embed_size, num_tokens, temperature, num_heads)

start_token = torch.tensor([[0]])
sequence_length = n_time_steps
memory = G.initial_state(batch_size = 1)
print(memory)
logits, tokens, _, memory = G(start_token, memory, sequence_length, None)
print(logits)
print(tokens)
print(memory)


tensor([[[1., 0.],
         [0., 1.],
         [0., 0.],
         [0., 0.]]])
tensor([[0.2129, 0.7301, 0.0570],
        [0.7978, 0.0577, 0.1445],
        [0.3692, 0.5117, 0.1191],
        [0.6053, 0.2216, 0.1731],
        [0.0462, 0.3125, 0.6413],
        [0.0271, 0.1808, 0.7921]], grad_fn=<CatBackward>)
tensor([[1],
        [0],
        [1],
        [0],
        [2],
        [2]])
tensor([[[ 1.4310, -1.4020],
         [ 1.3150,  0.9018],
         [ 0.8794, -1.1808],
         [ 0.8794, -1.1808]]], grad_fn=<AddBackward0>)


In [7]:
test = torch.tensor([[2]])
print(test.shape)
embed = nn.Embedding(10, 4)
test_embed = embed(test)
print(test_embed)
print(test_embed.shape)
test_inp = test_embed[:, 0]
print(test_inp.shape)
test_inp = test_inp.view(test_inp.shape[0], -1)
print(test_inp.shape)
test_inp.unsqueeze(dim=1).shape



torch.Size([1, 1])
tensor([[[-0.9160, -0.2775, -1.0579, -1.3390]]], grad_fn=<EmbeddingBackward>)
torch.Size([1, 1, 4])
torch.Size([1, 4])
torch.Size([1, 4])


torch.Size([1, 1, 4])

In [8]:
# Define the Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        pass